Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [ ]:
print(analysis)

In [3]:
ticker = ['MSFT']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

# prompt_start = f"This is your client. I want you to advice me on the stocks I list below.\
# 	I want to know the stocks' performance, if they're overvalued or undervalued, and whether or not they'll be a good investment.\
# 	I am looking for a long term (5+ years) investment and I have a relatively high risk tolerance.\
# 	Stocks: {ticker}"

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-9zvHlQ4LJFKmvTBrfkk34cep has been deleted
file name: df_stocks.csv is uploaded, new file id: file-DeTnJqQ1YLv3k9eIwtDM2gyF
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-H5S8gK32pv2p4h2GTlhZdgdA has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-S4Abs9yu7703O4krAD1u5uQh
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-HYNpois3DCTFKCwJaeI4OW1h has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-Gg611EJjwibKJ49ZYcGAETTf
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [4]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks, use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst,
	prompt=prompt_start,
	attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Here are the information gathered from the financial analyst:
These are the stocks the client wants advice on:
- MSFT (Microsoft Corporation)

I will analyze the data provided from the three uploaded files to offer insights on the performance of Microsoft's stock.

Let's start by loading and inspecting the contents of the files.
assistant > code_interpreter

import pandas as pd

# File paths
file1_path = '/mnt/data/file-Gg611EJjwibKJ49ZYcGAETTf'
file2_path = '/mnt/data/file-S4Abs9yu7703O4krAD1u5uQh'
file3_path = '/mnt/data/file-DeTnJqQ1YLv3k9eIwtDM2gyF'

# Load the data
file1 = pd.read_excel(file1_path)
file2 = pd.read_excel(file2_path)
file3 = pd.read_excel(file3_path)

# Display the first few rows to understand the content
file1_head = file1.head()
file2_head = file2.head()
file3_head = file3.head()

file1_head, file2_head, file3_head
assistant > It appears that the files could not be read as Excel files. Let me attempt reading them in a different format, possibly as CSV

In [48]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Sure! Here is a short sample response:

### Recommendation for MSFT
**Buy**.

### Summary:
- **Strong Growth**: Significant revenue and net income growth.
- **Robust Financials**: High free cash flow, substantial cash reserves.
- **Market Leader**: Leading position and innovation in key tech sectors.

### Key Metrics:
- **P/E Ratio**: 25.53 (indicating strong investor confidence)

Microsoft Corporation (MSFT) is a strong fit for a diversified, long-term growth portfolio.Image generated: file-x2DMkt5iGZYGudrpyGcdK3nA


In [ ]:
# thread.messages
type(client.beta.threads.messages.list(thread_id=thread.thread_id).data[0])

# type(file_cashflow)

In [5]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]
thread.messages

[{'message_id': 'msg_5tqyd1keiX2QhFyp11NtHcwt',
  'assistant_id': None,
  'created_at': 1725607691,
  'file_ids': [],
  'role': 'user',
  'run_id': None,
  'message_text': "This is the financial consultant.\tThe client wants advice on the following stocks.\tI need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\tYou will be provided with the data of these stocks, use them as you see fit.\tStocks: ['MSFT']"},
 {'message_id': 'msg_97STFxxTqb5X5iU7zO6C7KtE',
  'assistant_id': 'asst_Mqf9cO1sDTOd4UMYwcFfIQrA',
  'created_at': 1725607694,
  'file_ids': [],
  'role': 'assistant',
  'run_id': 'run_O1D6tY4oVvKx8iSZzeJpI0Gd',
  'message_text': "### Cash Flow Analysis\n\nBelow is the summary of the relevant metrics from the cash flow statement of Microsoft Corporation:\n\n| Date       | Net Income        | Change in Receivables | Change in Account Payable | Change in Inventory | Depreciation    | Amortization    | Capex           | Free Cash Flow 

In [6]:
thread.delete_thread()

thread: thread_aLACFYn2LjezFmaC23UZlUAz has been deleted.
